In [53]:
!pip install -r /content/sample_data/requirements.txt
#adicionei o matplotlib para graficos e o numpy para a porcentagem
!pip install matplotlib
!pip install numpy

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x7b4f5454dee0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/py4j/java_gateway.py", line 1357, in <lambda>
    lambda wr, cc=self._gateway_client, id=self._target_id:

KeyboardInterrupt: 


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
              ^^^^^^^^^

In [ ]:
#Transformando o arquivo excel dos dados_tic_kids_geral em CSV
#Nao consegui baixar os dados CSV do git, entao tive que baixar os dados excel no google drive
#E esse codigo cria um novo arquivo CSV com os dados do arquivo excel

import pandas as pd

excel_file = '/content/sample_data/dados_tic_kids_geral.xlsx'
csv_file = '/content/sample_data/dados_tic_kids_geral.csv'

read_file = pd.read_excel(excel_file)
read_file.to_csv(csv_file, index = None, header=True)


In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit, count, round as spark_round
spark = SparkSession.builder.master("local[*]").appName("Tic_Kids_Analise").getOrCreate()

dataset = spark.read.csv('../content/sample_data/dados_tic_kids_geral.csv', header=True, inferSchema=True, multiLine=True, escape='"')
# dataset.printSchema()

dataset_unificado = dataset.withColumn(
    "uso_rede_social",
    coalesce(
        col(dataset.columns[4]),)
)
# Vou utilizar apenas a 5ª Coluna por enquanto
# "Nos últimos três meses, a criança/adolescente mandou mensagens no Whatsapp, Snapchat ou chat do Facebook?"
#
#
#         col(dataset.columns[10]),
#         col(dataset.columns[12]),
#     )
# )

# dataset_unificado = dataset_unificado.withColumn(
#     "freq_rede_social",
#     coalesce(
#         col(dataset.columns[5]),
#         col(dataset.columns[7]),
#         col(dataset.columns[11]),
#         col(dataset.columns[13]),
#     )
# )



In [52]:
# tabela_UsoRedes_Série = dataset_unificado.select("ano", "uso_rede_social", "freq_rede_social", "Em que ano a criança/adolescente está na escola?").show(5, truncate=False)
dataset_filtrado = dataset_unificado.filter(
    col("Em que ano a criança/adolescente está na escola?").contains("Ensino Médio") &
    (col("Ano").cast("int") <= 2019)
)

total_alunos = dataset_filtrado.count()

df_contagem = dataset_filtrado.groupBy("uso_rede_social").agg(count(lit(1)).alias("contagem")
)
# groupby agrupa os valores da coluna em "equipes" de cada tipo de valor (sim,nao,nao se aplica), e o agg "agrupa", o lit cria uma
# coluna temporaria que tera 1 em todas as linhas, e o agg vai somar todas as linhas,
# assim teremos a contagem da quantidade de SIMs e NAOs

df_porcentagem = df_contagem.withColumn(
    "porcentagem",
    spark_round((col("contagem") / lit(total_alunos)) * 100, 2)
)


valores_desejados = ["Sim", "Não", "Não se aplica"]
df_porcentagem_final = df_porcentagem.filter(
    col("uso_rede_social").isin(valores_desejados)
).select(
    "uso_rede_social",
    "Porcentagem"
).orderBy(col("Porcentagem").desc())

tabela_UsoRedes_EnsinoMedio = dataset_filtrado.select("Ano", "Em que ano a criança/adolescente está na escola?", "uso_rede_social").show(10, truncate=False)

print(f"Total de alunos do Ensino Médio (Ano <= 2019): {total_alunos}")
print("\nPorcentagem de uso de mensagens (Whatsapp, Snapchat ou chat do Facebook):")
df_porcentagem_final.show()

+----+------------------------------------------------+---------------+
|Ano |Em que ano a criança/adolescente está na escola?|uso_rede_social|
+----+------------------------------------------------+---------------+
|2017|1º ano do Ensino Médio                          |Sim            |
|2017|1º ano do Ensino Médio                          |Sim            |
|2017|2º ano do Ensino Médio                          |Sim            |
|2017|2º ano do Ensino Médio                          |Sim            |
|2017|1º ano do Ensino Médio                          |Sim            |
|2017|1º ano do Ensino Médio                          |Sim            |
|2017|1º ano do Ensino Médio                          |Não            |
|2017|2º ano do Ensino Médio                          |Sim            |
|2017|1º ano do Ensino Médio                          |Sim            |
|2017|1º ano do Ensino Médio                          |Sim            |
+----+------------------------------------------------+---------

In [ ]:
spark.stop()